In [35]:
import pandas as pd
import numpy as np
import json
import re

# #1
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

# #2
import keras
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# #3
from keras.datasets import mnist
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [2]:
# Import the jsonlines file and then fill a dataframe with the jsonlines data.
# Due to the computation time down stream, only 50k rows will be used.
df = pd.read_json("categorized-comments.jsonl", lines = True, nrows = 50000)
df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [3]:
# Create a function that cleans (lowercase transformation, removal of special chars, etc.) our txt field row by 
# row using three regular expressions

def clean_data(data):
    data=data.lower()
    data=re.sub('&lt;/?.*?&gt;',' &lt;&gt', data)
    data=re.sub('\\d|\\W+|_',' ', data)
    data=re.sub('[^a-zA-Z]'," ", data)
    return data

In [4]:
# Create our list of english stopwords
stopwords = stopwords.words('english')

In [5]:
# Clean our txt field using our previously defined function
df['txt'] = df['txt'].apply(lambda x:clean_data(x))
df.head()

,cat,txt
0,sports,barely better than gabbert he was significantl...
1,sports,fuck the ducks and the angels but welcome to a...
2,sports,should have drafted more wrs matt millen probably
3,sports,done https i imgur com yz pm jpg
4,sports,no noo


In [6]:
'''
#1 

Fit a neural network classifier using sklearn and report the model accuracy, 
percision, f1 score, and the confusion matrix.

'''

'\n#1 \n\nFit a neural network classifier using sklearn and report the model accuracy, \npercision, f1 score, and the confusion matrix.\n\n'

In [7]:
# Instantiate a count vectorizer while ignoring the stopwords
cv = CountVectorizer(stop_words = stopwords)

# Fit our count vectorizer with our txt field and save it as our input variable for our neural network classifier
X = cv.fit_transform(df['txt'])
Y = df['cat']

# Split our data set for training and testing purposes
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=1)

In [8]:
# Instantiate our multi-layer perceptron classifier with 100 hidden layers
mlp = MLPClassifier(hidden_layer_sizes=10, verbose = True)

In [9]:
# Fit our neural network classifier with our training data
mlp.fit(X_train,y_train)

Iteration 1, loss = 0.54078378
Iteration 2, loss = 0.33902826
Iteration 3, loss = 0.26358546
Iteration 4, loss = 0.22303336
Iteration 5, loss = 0.19672773
Iteration 6, loss = 0.17768878
Iteration 7, loss = 0.16306179
Iteration 8, loss = 0.15143619
Iteration 9, loss = 0.14202464
Iteration 10, loss = 0.13419728
Iteration 11, loss = 0.12773423
Iteration 12, loss = 0.12206232
Iteration 13, loss = 0.11728928
Iteration 14, loss = 0.11299526
Iteration 15, loss = 0.10943814
Iteration 16, loss = 0.10618278
Iteration 17, loss = 0.10348423
Iteration 18, loss = 0.10089233
Iteration 19, loss = 0.09874630
Iteration 20, loss = 0.09672449
Iteration 21, loss = 0.09493408
Iteration 22, loss = 0.09331358
Iteration 23, loss = 0.09187328
Iteration 24, loss = 0.09053142
Iteration 25, loss = 0.08930152
Iteration 26, loss = 0.08815980
Iteration 27, loss = 0.08711917
Iteration 28, loss = 0.08627593
Iteration 29, loss = 0.08546292
Iteration 30, loss = 0.08459402
Iteration 31, loss = 0.08381975
Iteration 32, los

MLPClassifier(hidden_layer_sizes=10, verbose=True)

In [10]:
# Create a set of predictions using our testing data
preds = mlp.predict(X_test)

In [11]:
print('Model Accuracy: \n', accuracy_score(y_test, preds))

Model Accuracy: 
 0.8388


In [12]:
print('Confusion Matrix: \n', confusion_matrix(y_test, preds)) 

Confusion Matrix: 
 [[8700 1424]
 [1800 8076]]


In [13]:
print('Classification Report: \n', classification_report(y_test, preds)) 

Classification Report: 
                         precision    recall  f1-score   support

science_and_technology       0.83      0.86      0.84     10124
                sports       0.85      0.82      0.83      9876

              accuracy                           0.84     20000
             macro avg       0.84      0.84      0.84     20000
          weighted avg       0.84      0.84      0.84     20000



In [14]:
# Clear the memory of all large variables
del df
del X_train
del X_test
del y_train
del y_test
del preds

In [15]:
'''
#2

Fit a neural network classifier using keras and report the model accuracy, 
percision, f1 score, and the confusion matrix.

'''

'\n#2\n\nFit a neural network classifier using keras and report the model accuracy, \npercision, f1 score, and the confusion matrix.\n\n'

In [16]:
# Import the categorized comments into a dataframe

df = pd.read_json("categorized-comments.jsonl", lines = True, nrows = 100000)
df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [17]:
# Define the sample size
size = 1500

# Define the replace argument in our lambda function
replace = True

# Define the lambda function that randomly chooses 1500 of each category for our resulting dataframe
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]

# Group each of our categories in a new dataset and apply our previously defined lambda function
cat_df = df.groupby('cat', as_index=False).apply(fn)

# Remove all punctuation, make all letters lowercase,and remove special characters
cat_df['txt'] = cat_df['txt'].apply(lambda x:clean_data(x))

# Reset the index of our cleaned dataframe
cat_df.reset_index(drop=True, inplace=True)

cat_df.head()

,cat,txt
0,science_and_technology,does this work if you use gravitybox to flip t...
1,science_and_technology,would be also cool if they came without bootlo...
2,science_and_technology,but it does
3,science_and_technology,i love allo s stickers maybe allo could be the...
4,science_and_technology,gt as you know nougat has been out for a dece...


In [18]:
# Verify that the categories have equal row counts
cat_df.groupby(["cat"]).size()

cat
science_and_technology    1500
sports                    1500
video_games               1500
dtype: int64

In [19]:
# Instantiate the LabelEncoder
# The label encoder is a type of encoding that should only be used on the target values
# This replaces our categorical data with numeric data that ranges from 0 to n - 1.
enc = LabelEncoder()

# Define our target variable
cat = cat_df["cat"]

# Fit our encoder and return a new row of numeric data
cat_df["cat"] = enc.fit_transform(cat)

In [20]:
# Verify that the encoding worked as expected
# We are expecting the same dataframe that we saw the previous time we ran this code, except that the categories are numeric
cat_df.groupby(["cat"]).size()

cat
0    1500
1    1500
2    1500
dtype: int64

In [21]:
# Define the max number of features our count vectorizer will use
feat_count = 1500

# Instantiate the count vectorizer, which we will in turn use to create a feature matrix
cv = CountVectorizer(analyzer='word',
                     stop_words=stopwords, 
                     max_features = feat_count,
                     max_df = 0.5,
                     min_df = 3)

# Define a feature matrix based on our txt field
X = cv.fit_transform(cat_df['txt'])

# Define our target variable
y = cat_df['cat']

# Split our dataset for training and testing purposes using 40% of our data for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [22]:
# Convert a sparse matrix to a sparse tensor so that the sequential classifier can accept it
def convert_sparse_matrix_to_sparse_tensor(X):
    # Create a coordinate representation of our scipy sparse matrix
    coo = X.tocoo()
    
    # Define the indicies of the coordinate matrix and transpose the matrix
    indices = np.mat([coo.row, coo.col]).transpose()
    
    # Return a sparse tensor
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [23]:
# Transform our scipy sparse matrix to a sparse tensor and then reorder the indicies
X_train = tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(X_train))

In [24]:
# Convert the pandas series to a tensor
y_train = tf.convert_to_tensor(y_train)

In [25]:
# Instantiate our sequential classifier
classifier_seq = Sequential()

# Add layers to our sequential classifier
classifier_seq.add(Dense(units=500,activation="relu",input_shape=(feat_count,)))
classifier_seq.add(Dense(units=50, activation="relu"))
classifier_seq.add(Dense(units=4, activation="softmax"))

# Compile the sequential artificial neural network
classifier_seq.compile(optimizer="rmsprop", 
                       loss="sparse_categorical_crossentropy", 
                       metrics=["accuracy"])

In [26]:
# Fit the model using the training data and expose the progress of the fit with the verbose argument
classifier_seq.fit(X_train, y_train, batch_size=200, epochs=5, verbose = 1)

Epoch 1/5
14/14 [==============================] - 0s 6ms/step - loss: 1.1903 - accuracy: 0.4724
Epoch 2/5
14/14 [==============================] - 0s 6ms/step - loss: 0.7111 - accuracy: 0.7913
Epoch 3/5
14/14 [==============================] - 0s 6ms/step - loss: 0.5159 - accuracy: 0.8269
Epoch 4/5
14/14 [==============================] - 0s 6ms/step - loss: 0.3847 - accuracy: 0.8583
Epoch 5/5
14/14 [==============================] - 0s 6ms/step - loss: 0.3181 - accuracy: 0.8757


In [27]:
# Observe a summary of the layers of our neural network
classifier_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               750500    
_________________________________________________________________
dense_1 (Dense)              (None, 50)                25050     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 204       
Total params: 775,754
Trainable params: 775,754
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Transform our scipy sparse matrix to a sparse tensor and then reorder the indicies
X_test = tf.sparse.reorder(convert_sparse_matrix_to_sparse_tensor(X_test))

In [29]:
# Convert the pandas series to a tensor
y_test = tf.convert_to_tensor(y_test)

In [30]:
# Evaluate the model and define loss and accuracy
loss, accuracy = classifier_seq.evaluate(X_test, y_test, verbose=1)
print("Training Accuracy: \n".format(accuracy))

57/57 [==============================] - 0s 652us/step - loss: 0.6153 - accuracy: 0.7228
Training Accuracy: 



In [31]:
# Define the target predictions based upon on the test set of our inputs
y_pred = classifier_seq.predict_classes(X_test)

G:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [32]:
# Print the metrics of our model
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("Classification Report: \n", classification_report(y_test,y_pred))
print("Accuracy: \n", accuracy_score(y_test,y_pred))

Confusion Matrix: 
 [[454 129  30]
 [ 68 460  44]
 [ 63 165 387]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.78      0.74      0.76       613
           1       0.61      0.80      0.69       572
           2       0.84      0.63      0.72       615

    accuracy                           0.72      1800
   macro avg       0.74      0.72      0.72      1800
weighted avg       0.74      0.72      0.72      1800

Accuracy: 
 0.7227777777777777


In [33]:
# Clear the memory of our larger variables
del df
del X_train
del X_test
del y_train
del y_test
del y_pred

In [34]:
'''
#3

Classify MSINT images using convolutional neural network and report the accuracy of the results.

'''

'\n#3\n\nClassify MSINT images using convolutional neural network and report the accuracy of the results.\n\n'

In [36]:
# Set the color channel value 
K.set_image_data_format("channels_first")

# Set seed
np.random.seed(0)

In [37]:
# Set image information
channels = 1
height = 28
width = 28

In [38]:
# Load the data and target from MNIST data
(data_train, target_train), (data_test, target_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [39]:
# Reshape the training data image data into features
data_train = data_train.reshape(data_train.shape[0], channels, height, width)

# Reshape the training data image data into features
data_test = data_test.reshape(data_test.shape[0], channels, height, width)

In [40]:
# Rescale the pixel intensity to between 0 and 1
features_train = data_train / 255
features_test = data_test / 255

In [41]:
# One-hot encode target
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_train.shape[1]

In [42]:
# Start sequential neural network 
network = Sequential()

In [44]:
# Add convolutional layer with 64 filters, a 5x5 window, and ReLU activation function
network.add(Conv2D(filters = 64,
                  kernel_size = (5,5),
                  input_shape = (channels, width, height),
                  activation = 'relu'))

In [45]:
# Add max pooling layer with a 2x2 layer window
network.add(MaxPooling2D(pool_size=(2,2)))

In [46]:
# Add dropout layer
network.add(Dropout(0.5))

In [47]:
# Add a layer to flatten the input
network.add(Flatten())

In [54]:
# Add fully connected layer of 128 units with a ReLU activation function
network.add(Dense(10, activation="relu"))

In [55]:
# Add another dropout layer
network.add(Dropout(0.5))

In [56]:
# Compile the sequential neural network
network.compile(loss = "categorical_crossentropy",
               optimizer = "rmsprop",
               metrics = ["accuracy"])

In [57]:
# Train the sequential neural network
network.fit(features_train,
           target_train,
           epochs = 2,
           verbose = 0,
           batch_size = 1000,
           validation_data = (features_test, target_test))

InvalidArgumentError:  Default MaxPoolingOp only supports NHWC on device type CPU
	 [[node sequential_1/max_pooling2d/MaxPool (defined at <ipython-input-57-6a17f0cd7ec4>:2) ]] [Op:__inference_train_function_2338]

Function call stack:
train_function
